<a href="https://colab.research.google.com/github/kenzhou97/examples/blob/master/MultiChoiceSwag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [39]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [84]:
import logging
# logging.root.setLevel(logging.WARN)# not working
# logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("main")
logger.setLevel(logging.INFO)

In [7]:
!pip install transformers

     |████████████████████████████████| 675kB 2.8MB/s 
     |████████████████████████████████| 3.8MB 13.9MB/s 
     |████████████████████████████████| 1.1MB 31.3MB/s 
     |████████████████████████████████| 890kB 30.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=08f5fbfffe1b82f1cd02112add51f3990ba0cd4a2a1be47da5b712f86c0a4111
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [8]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=c0437240d14251807dd92f5e3446154e6bc711638d3212a2c293274808060711
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [9]:
!pip install kaggle-cli

     |████████████████████████████████| 81kB 2.2MB/s 
     |████████████████████████████████| 5.3MB 5.5MB/s 
     |████████████████████████████████| 122kB 36.5MB/s 
     |████████████████████████████████| 112kB 38.9MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
  Created wheel for kaggle-cli: filename=kaggle_cli-0.12.13-cp36-none-any.whl size=9488 sha256=f035e159cec77363686eb13fe70b637747b34f253766770b7de18546cedbfe29
  Stored in directory: /root/.cache/pip/wheels/d5/bb/10/c1dd1b08c7433c943cb55c46367ae3f891415e8a37300ff8a7
  Created wheel for pyperclip: filename=pyperclip-1.8.0-cp36-none-any.whl size=8693 sha256=95c17c49ac6854c36fc98e77d8a2377fee39a00bd78a4bba64f1903353b382a5
  Stored in directory: /root/.cache/pip/wheels/b2/ac/0a/b784f0afe26eaf52e88a7e15c7369090deea0354fa1c6fc689
Successfully built kaggle-cli pyperclip
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
  Found existing installation: progressbar

In [10]:
# !kaggle datasets list # https://www.kaggle.com/c/two-sigma-financial-news/discussion/83593

In [11]:
# import wget
# import os

# print('Downloading dataset...')

# # The URL for the *training* examples.
# url = 'https://www.kaggle.com/jeromeblanchet/swag-nlp-dataset/download'

# # Download the file (if we haven't already)
# #if not os.path.exists('./restauranttrain.bio'):
# wget.download(url, './swag.zip')

In [13]:
# upload swag 
from google.colab import files
from os import path
if not path.exists("./swag.zip"):
  uploaded = files.upload()

Saving swag.zip to swag.zip


In [14]:
import zipfile
with zipfile.ZipFile("./swag.zip","r") as z:
    z.extractall("./swag/")

In [15]:
!ls -l ./swag

total 82520
-rw-r--r-- 1 root root  7817885 Jun 21 17:13 test.csv
-rw-r--r-- 1 root root 28243333 Jun 21 17:13 train.csv
-rw-r--r-- 1 root root 31608559 Jun 21 17:13 train_full.csv
-rw-r--r-- 1 root root  7893588 Jun 21 17:13 val.csv
-rw-r--r-- 1 root root  8929065 Jun 21 17:13 val_full.csv


In [16]:
import pandas as pd
pd.read_csv('./swag/train.csv')

,Unnamed: 0,video-id,fold-ind,startphrase,sent1,sent2,gold-source,ending0,ending1,ending2,ending3,label
0,0,anetv_jkn6uvmqwh4,3416,Members of the procession walk down the street...,Members of the procession walk down the street...,A drum line,gold,passes by walking down the street playing thei...,has heard approaching them.,arrives and they're outside dancing and asleep.,turns the lead singer watches the performance.,0
1,1,anetv_jkn6uvmqwh4,3417,A drum line passes by walking down the street ...,A drum line passes by walking down the street ...,Members of the procession,gen,are playing ping pong and celebrating one left...,wait slowly towards the cadets.,continues to play as well along the crowd alon...,"continue to play marching, interspersed.",3
2,2,anetv_jkn6uvmqwh4,3415,A group of members in green uniforms walks wav...,A group of members in green uniforms walks wav...,Members of the procession,gold,pay the other coaches to cheer as people this ...,walk down the street holding small horn brass ...,is seen in the background.,are talking a couple of people playing a game ...,1
3,3,anetv_jkn6uvmqwh4,3417,A drum line passes by walking down the street ...,A drum line passes by walking down the street ...,Members of the procession,gen,are playing ping pong and celebrating one left...,wait slowly towards the cadets.,makes a square call and ends by jumping down i...,play and go back and forth hitting the drums w...,3
4,4,anetv_Bri_myFFu4A,2408,The person plays a song on the violin. The man,The person plays a song on the violin.,The man,gold,finishes the song and lowers the instrument.,hits the saxophone and demonstrates how to pro...,finishes massage the instrument again and cont...,continues dancing while the man gore the music...,0
...,...,...,...,...,...,...,...,...,...,...,...,...
73541,73541,lsmdc3073_THE_GUILT_TRIP-34734,2708,Someone bites her lower lip. Someone,Someone bites her lower lip.,Someone,gen,shifts her thoughtful gaze to the page.,sits down next to her on the bed.,"pats chin, looks down.",gives him a disapproving look.,1
73542,73542,anetv_R2izNBHhctM,62,The cat looked up while the woman is clipping ...,The cat looked up while the woman is clipping ...,"While the woman is clipping the left back paw,...",gen,appears on the left youth.,is overlaid on the nail.,zooms out holding her toothbrush.,stands in her lap as it moves away from her le...,3
73543,73543,anetv_R2izNBHhctM,60,A woman in pink dress is clipping the right pa...,A woman in pink dress is clipping the right pa...,"She took the left paw to clip, the cat looked ...",gold,is brushing on the wall with two toys.,gave them a bit and placed unto the cat's back.,looked down and gave her a quick kiss on the h...,drops the smooths it on the other side of the ...,2
73544,73544,anetv_R2izNBHhctM,62,The cat looked up while the woman is clipping ...,The cat looked up while the woman is clipping ...,"While the woman is clipping the left back paw,...",gold,was blinking to the camera.,is overlaid on the nail.,appears on the left youth.,zooms out holding her toothbrush.,0


In [17]:
class SwagExample(object):
    """A single training/test example for the SWAG dataset."""

    def __init__(self, swag_id, context_sentence, start_ending, ending_0, ending_1, ending_2, ending_3, label=None):
        self.swag_id = swag_id
        self.context_sentence = context_sentence
        self.start_ending = start_ending
        self.endings = [
            ending_0,
            ending_1,
            ending_2,
            ending_3,
        ]
        self.label = label

    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        attributes = [
            "swag_id: {}".format(self.swag_id),
            "context_sentence: {}".format(self.context_sentence),
            "start_ending: {}".format(self.start_ending),
            "ending_0: {}".format(self.endings[0]),
            "ending_1: {}".format(self.endings[1]),
            "ending_2: {}".format(self.endings[2]),
            "ending_3: {}".format(self.endings[3]),
        ]

        if self.label is not None:
            attributes.append("label: {}".format(self.label))

        return ", ".join(attributes)

In [18]:
import csv
def read_swag_examples(input_file, is_training=True):
    with open(input_file, "r", encoding="utf-8") as f:
        lines = list(csv.reader(f))

    if is_training and lines[0][-1] != "label":
        raise ValueError("For training, the input file must contain a label column.")

    examples = [
        SwagExample(
            swag_id=line[2],
            context_sentence=line[4],
            start_ending=line[5],  # in the swag dataset, the
            # common beginning of each
            # choice is stored in "sent2".
            ending_0=line[7],
            ending_1=line[8],
            ending_2=line[9],
            ending_3=line[10],
            label=int(line[11]) if is_training else None,
        )
        for line in lines[1:]  # we skip the line with the column names
    ]

    return examples

In [19]:
class InputFeatures(object):
    def __init__(self, example_id, choices_features, label):
        self.example_id = example_id
        self.choices_features = [
            {"input_ids": input_ids, "input_mask": input_mask, "segment_ids": segment_ids}
            for _, input_ids, input_mask, segment_ids in choices_features
        ]
        self.label = label

In [20]:
from tqdm import tqdm, trange
def convert_examples_to_features(examples, tokenizer, max_seq_length, is_training):
    """Loads a data file into a list of `InputBatch`s."""

    # Swag is a multiple choice task. To perform this task using Bert,
    # we will use the formatting proposed in "Improving Language
    # Understanding by Generative Pre-Training" and suggested by
    # @jacobdevlin-google in this issue
    # https://github.com/google-research/bert/issues/38.
    #
    # Each choice will correspond to a sample on which we run the
    # inference. For a given Swag example, we will create the 4
    # following inputs:
    # - [CLS] context [SEP] choice_1 [SEP]
    # - [CLS] context [SEP] choice_2 [SEP]
    # - [CLS] context [SEP] choice_3 [SEP]
    # - [CLS] context [SEP] choice_4 [SEP]
    # The model will output a single value for each input. To get the
    # final decision of the model, we will run a softmax over these 4
    # outputs.
    features = []
    for example_index, example in tqdm(enumerate(examples)):
        context_tokens = tokenizer.tokenize(example.context_sentence)
        start_ending_tokens = tokenizer.tokenize(example.start_ending)

        choices_features = []
        for ending_index, ending in enumerate(example.endings):
            # We create a copy of the context tokens in order to be
            # able to shrink it according to ending_tokens
            context_tokens_choice = context_tokens[:]
            ending_tokens = start_ending_tokens + tokenizer.tokenize(ending)
            # Modifies `context_tokens_choice` and `ending_tokens` in
            # place so that the total length is less than the
            # specified length.  Account for [CLS], [SEP], [SEP] with
            # "- 3"
            _truncate_seq_pair(context_tokens_choice, ending_tokens, max_seq_length - 3)

            tokens = ["[CLS]"] + context_tokens_choice + ["[SEP]"] + ending_tokens + ["[SEP]"]
            segment_ids = [0] * (len(context_tokens_choice) + 2) + [1] * (len(ending_tokens) + 1)

            input_ids = tokenizer.convert_tokens_to_ids(tokens)
            input_mask = [1] * len(input_ids)

            # Zero-pad up to the sequence length.
            padding = [0] * (max_seq_length - len(input_ids))
            input_ids += padding
            input_mask += padding
            segment_ids += padding

            assert len(input_ids) == max_seq_length
            assert len(input_mask) == max_seq_length
            assert len(segment_ids) == max_seq_length

            choices_features.append((tokens, input_ids, input_mask, segment_ids))

        label = example.label
        if example_index < 5:
            logger.info("*** Example ***")
            logger.info("swag_id: {}".format(example.swag_id))
            for choice_idx, (tokens, input_ids, input_mask, segment_ids) in enumerate(choices_features):
                logger.info("choice: {}".format(choice_idx))
                logger.info("tokens: {}".format(" ".join(tokens)))
                logger.info("input_ids: {}".format(" ".join(map(str, input_ids))))
                logger.info("input_mask: {}".format(" ".join(map(str, input_mask))))
                logger.info("segment_ids: {}".format(" ".join(map(str, segment_ids))))
            if is_training:
                logger.info("label: {}".format(label))

        features.append(InputFeatures(example_id=example.swag_id, choices_features=choices_features, label=label))

    return features

In [21]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [22]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [23]:
from tqdm import tqdm, trange
def convert_examples_to_features_better(examples, tokenizer, max_seq_length, is_training):
    """Loads a data file into a list of `InputBatch`s."""

    # Swag is a multiple choice task. To perform this task using Bert,
    # we will use the formatting proposed in "Improving Language
    # Understanding by Generative Pre-Training" and suggested by
    # @jacobdevlin-google in this issue
    # https://github.com/google-research/bert/issues/38.
    #
    # Each choice will correspond to a sample on which we run the
    # inference. For a given Swag example, we will create the 4
    # following inputs:
    # - [CLS] context [SEP] choice_1 [SEP]
    # - [CLS] context [SEP] choice_2 [SEP]
    # - [CLS] context [SEP] choice_3 [SEP]
    # - [CLS] context [SEP] choice_4 [SEP]
    # The model will output a single value for each input. To get the
    # final decision of the model, we will run a softmax over these 4
    # outputs.
    features = []
    for example_index, example in tqdm(enumerate(examples), mininterval=1000):
      context_sentence = example.context_sentence
      start_ending = example.start_ending
      logger.debug(f'{example.swag_id} context_sentence - {context_sentence}')
      choices_features = []
      for ending_index, ending in enumerate(example.endings):
        ending_sentence = " ".join([start_ending, ending])
        logger.debug(f'{example.swag_id} ending_sentence - {ending_sentence}')
        encodings = tokenizer.encode_plus(context_sentence, ending_sentence, add_special_tokens=True, max_length=max_seq_length, pad_to_max_length=True, truncation_strategy='longest_first')
        input_ids = encodings['input_ids']
        token_type_ids = encodings['token_type_ids']
        attention_mask = encodings['attention_mask']
        # print(type(input_ids))
        # print(input_ids[0])
        input_tokens = tokenizer.convert_ids_to_tokens(input_ids)
        input_tokens_str = ' '.join(input_tokens)

        assert len(input_ids) == max_seq_length
        assert len(attention_mask) == max_seq_length
        assert len(token_type_ids) == max_seq_length

        choices_features.append((input_tokens, input_ids, attention_mask, token_type_ids))

      label = example.label
      features.append(InputFeatures(example_id=example.swag_id, choices_features=choices_features, label=label))

      if example_index < 3:
        for choice_idx, (input_tokens, input_ids, attention_mask, token_type_ids) in enumerate(choices_features):
          if choice_idx == 0:
            logger.info("*** Example ***")
            logger.info("swag_id: {}".format(example.swag_id))
          logger.info(f'choice_idx - {choice_idx}')
          logger.info(f'input_tokens - {input_tokens_str}')
          logger.info(f'input_ids - {input_ids}')
          logger.info(f'token_type_ids - {token_type_ids}')
          logger.info(f'attention_mask - {attention_mask}')
          if is_training:
            logger.info(f'label - {label}')

    return features

In [24]:
from tqdm import tqdm, trange
def convert_examples_to_features_better_tensor(examples, tokenizer, max_seq_length, is_training):
    """Loads a data file into a list of `InputBatch`s."""

    # Swag is a multiple choice task. To perform this task using Bert,
    # we will use the formatting proposed in "Improving Language
    # Understanding by Generative Pre-Training" and suggested by
    # @jacobdevlin-google in this issue
    # https://github.com/google-research/bert/issues/38.
    #
    # Each choice will correspond to a sample on which we run the
    # inference. For a given Swag example, we will create the 4
    # following inputs:
    # - [CLS] context [SEP] choice_1 [SEP]
    # - [CLS] context [SEP] choice_2 [SEP]
    # - [CLS] context [SEP] choice_3 [SEP]
    # - [CLS] context [SEP] choice_4 [SEP]
    # The model will output a single value for each input. To get the
    # final decision of the model, we will run a softmax over these 4
    # outputs.
    input_ids_all = []
    token_type_ids_all = []
    attention_mask_all = []
    labels_all = []
    for example_index, example in tqdm(enumerate(examples), mininterval=1000):
      context_sentence = example.context_sentence
      start_ending = example.start_ending
      logger.debug(f'{example.swag_id} context_sentence - {context_sentence}')
      input_tokens_item = []
      input_ids_item = []
      token_type_ids_item = []
      attention_mask_item = []
      for ending_index, ending in enumerate(example.endings):
        ending_sentence = " ".join([start_ending, ending])
        logger.debug(f'{example.swag_id} ending_sentence - {ending_sentence}')
        encodings = tokenizer.encode_plus(context_sentence, ending_sentence, add_special_tokens=True, max_length=max_seq_length, pad_to_max_length=True, truncation_strategy='longest_first', return_tensors='pt')
        input_ids = encodings['input_ids'][0]
        token_type_ids = encodings['token_type_ids'][0]
        attention_mask = encodings['attention_mask'][0]
        # print(type(input_ids))
        # print(input_ids[0])
        input_tokens = tokenizer.convert_ids_to_tokens(input_ids)
        input_tokens_str = ' '.join(input_tokens)

        assert len(input_ids) == max_seq_length
        assert len(attention_mask) == max_seq_length
        assert len(token_type_ids) == max_seq_length

        input_tokens_item.append(input_tokens_str)
        input_ids_item.append(input_ids)
        token_type_ids_item.append(token_type_ids)
        attention_mask_item.append(attention_mask)

      label = example.label

      input_ids_all.append(torch.stack(input_ids_item, dim=0))
      token_type_ids_all.append(torch.stack(token_type_ids_item, dim=0))
      attention_mask_all.append(torch.stack(attention_mask_item, dim=0))
      labels_all.append(label)

      if example_index < 1:
        for choice_idx, (input_tokens, input_ids, token_type_ids, attention_mask) in enumerate(zip(input_tokens_item, input_ids_item, token_type_ids_item, attention_mask_item)):
          if choice_idx == 0:
            logger.info("*** Example ***")
            logger.info("swag_id: {}".format(example.swag_id))
          logger.info(f'choice_idx - {choice_idx}')
          logger.info(f'input_tokens - {input_tokens}')
          logger.info(f'input_ids - {input_ids}')
          logger.info(f'token_type_ids - {token_type_ids}')
          logger.info(f'attention_mask - {attention_mask}')
          if is_training: # ???
            logger.info(f'label - {label}')

    return input_ids_all, token_type_ids_all, attention_mask_all, labels_all


In [25]:
!ls

sample_data  swag  swag.zip


In [26]:
max_seq_length = 64
is_training = True

In [142]:
examples = read_swag_examples("./swag/train.csv")
logger.info(f'Total {len(examples)} examples.')

INFO:main:Total 73546 examples.


In [143]:
examples = examples[:10000]

In [144]:
# # Old approach
# features = convert_examples_to_features_better(examples, tokenizer, max_seq_length, is_training=is_training)
# def select_field(features, field):
#     return [[choice[field] for choice in feature.choices_features] for feature in features]
# all_input_ids = torch.tensor(select_field(features, "input_ids"), dtype=torch.long)


In [145]:
# features = convert_examples_to_features_better(examples, tokenizer, max_seq_length, is_training=is_training)
input_ids_all, token_type_ids_all, attention_mask_all, labels_all = convert_examples_to_features_better_tensor(examples, tokenizer, max_seq_length, is_training=is_training)

0it [00:00, ?it/s]INFO:main:*** Example ***
INFO:main:swag_id: 3416
INFO:main:choice_idx - 0
INFO:main:input_tokens - [CLS] members of the procession walk down the street holding small horn brass instruments . [SEP] a drum line passes by walking down the street playing their instruments . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:main:input_ids - tensor([  101,  2372,  1997,  1996, 14385,  3328,  2091,  1996,  2395,  3173,
         2235,  7109,  8782,  5693,  1012,   102,  1037,  6943,  2240,  5235,
         2011,  3788,  2091,  1996,  2395,  2652,  2037,  5693,  1012,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
          

In [146]:
# stackfeatures_tensor.choices_features

In [147]:
!ls

sample_data  swag  swag.zip


In [148]:
# print(input_ids_all)
input_ids_all_pt = torch.stack(input_ids_all, dim=0)
token_type_ids_all_pt = torch.stack(token_type_ids_all, dim=0)
attention_mask_all_pt = torch.stack(attention_mask_all, dim=0)
labels_all_pt = torch.tensor(labels_all, dtype=torch.long)

In [149]:
input_ids_all_pt.size()
labels_all_pt.size()

torch.Size([10000])

In [150]:
from torch.utils.data import TensorDataset

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids_all_pt, token_type_ids_all_pt, attention_mask_all_pt, labels_all_pt)

In [151]:
def set_seed(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    # n_gpu > 0:
    torch.cuda.manual_seed_all(seed_val)

In [152]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
import random
import numpy as np

def train(model, dataset, batch_size, epochs, learning_rate, adam_epsilon, warmup_steps):
    """ Train the model """

    dataloader = DataLoader(dataset, sampler=RandomSampler(dataset), batch_size=batch_size)
    total_steps = len(dataloader) * epochs

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    # optimizer_grouped_parameters = [
    #     {
    #         "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
    #         "weight_decay": weight_decay,
    #     },
    #     {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    # ]
    # optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)
    optimizer = AdamW(model.parameters(), lr=learning_rate, eps=adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps
    )

    # Ignored distributed n-gpu training

    # Train!
    set_seed(42)  # Added here for reproductibility
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(dataset))
    logger.info("  Num Epochs = %d", epochs)
    logger.info("  Batch size = %d", batch_size)

    loss_values = []
    train_iterator = trange(epochs, desc="Epoch")
    for _ in train_iterator:
        model.train() # move out?
        epoch_iterator = tqdm(dataloader, desc="Iteration")
        epoch_total_loss = 0.0
        for step, batch in enumerate(epoch_iterator):
            input_ids = batch[0].to(device)
            token_type_ids = batch[1].to(device)
            attention_mask = batch[2].to(device)
            labels = batch[3].to(device)

            outputs = model(input_ids, 
                    token_type_ids=token_type_ids, 
                    attention_mask=attention_mask, 
                    labels=labels)
            # outputs = model(**inputs)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)
            epoch_total_loss += loss.item()

            
            loss.backward()
            max_grad_norm = 1.0
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

            # Removed save checkpoint
            # Removed gradient accumulation steps
            optimizer.step()
            scheduler.step()
            model.zero_grad()

        epoch_avg_loss = epoch_total_loss / len(dataloader) # has issue, where is epoch?
        loss_values.append(epoch_avg_loss)

    return loss_values

In [153]:
# Train
from transformers import BertForMultipleChoice
model = BertForMultipleChoice.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 1, # The number of output labels--18 for our NER dataset
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
model.to(device)

batch_size = 32
epochs = 3
learning_rate = 5e-5
adam_epsilon = 1e-8
warmup_steps = 0
loss_values = train(model, dataset, batch_size, epochs, learning_rate, adam_epsilon, warmup_steps)
logger.info(f'Average loss = {loss_values}')

INFO:main:***** Running training *****
INFO:main:  Num examples = 10000
INFO:main:  Num Epochs = 3
INFO:main:  Batch size = 32
Epoch: 100%|██████████| 3/3 [35:28<00:00, 709.36s/it]
INFO:main:Average loss = [0.9039130133752245, 0.3785224791866141, 0.1257126528930407]


In [154]:
logging.basicConfig(level=logging.INFO)
logger.info(f'Average loss = {loss_values}')

INFO:main:Average loss = [0.9039130133752245, 0.3785224791866141, 0.1257126528930407]


In [155]:
print(f" Average loss = {loss_values}")

 Average loss = [0.9039130133752245, 0.3785224791866141, 0.1257126528930407]


In [156]:
# Evaluation
# TODO validation with train?
test_examples = read_swag_examples("./swag/val.csv")
logger.info(f'Total {len(test_examples)} test examples.')
test_examples = test_examples[: 1000]

INFO:main:Total 20006 test examples.


In [157]:
input_ids_all_test, token_type_ids_all_test, attention_mask_all_test, labels_all_test = convert_examples_to_features_better_tensor(test_examples, tokenizer, max_seq_length, is_training=False)

0it [00:00, ?it/s]INFO:main:*** Example ***
INFO:main:swag_id: 18313
INFO:main:choice_idx - 0
INFO:main:input_tokens - [CLS] students lower their eyes nervously . [SEP] she pat ##s her shoulder , then sa ##unt ##ers toward someone . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:main:input_ids - tensor([  101,  2493,  2896,  2037,  2159, 12531,  1012,   102,  2016,  6986,
         2015,  2014,  3244,  1010,  2059,  7842, 16671,  2545,  2646,  2619,
         1012,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     

In [158]:
input_ids_all_pt_test = torch.stack(input_ids_all_test, dim=0)
token_type_ids_all_pt_test = torch.stack(token_type_ids_all_test, dim=0)
attention_mask_all_pt_test = torch.stack(attention_mask_all_test, dim=0)
labels_all_pt_test = torch.tensor(labels_all_test, dtype=torch.long)
dataset_test = TensorDataset(input_ids_all_pt_test, token_type_ids_all_pt_test, attention_mask_all_pt_test, labels_all_pt_test)

In [159]:
from torch.utils.data import SequentialSampler

def test(model, dataset, batch_size):
    # Note that DistributedSampler samples randomly
    dataloader = DataLoader(dataset, sampler=SequentialSampler(dataset), batch_size=batch_size)

    # Eval!
    logger.info("  Num examples = %d", len(dataset))
    logger.info("  Batch size = %d", batch_size)

    predicted_logits = []
    true_labels = []
    eval_loss = 0.0

    model.eval()
    for batch in tqdm(dataloader, desc="Evaluating"):
        batch = tuple(t.to(device) for t in batch)
        input_ids, token_type_ids, attention_mask, labels = batch
        with torch.no_grad():
            outputs = model(input_ids, 
                    token_type_ids=token_type_ids, 
                    attention_mask=attention_mask, 
                    labels=labels)
            eval_loss_batch, logits = outputs[:2]
            eval_loss += eval_loss_batch.mean().item() # why mean, dimension?

        logits = logits.detach().cpu().numpy() # type?
        label_ids = labels.to("cpu").numpy()
        
        predicted_logits.append(logits)
        true_labels.append(label_ids)

    eval_loss = eval_loss / len(dataloader)

    return eval_loss, predicted_logits, true_labels

In [160]:
eval_loss, predicted_logits, true_labels = test(model, dataset_test, batch_size)

INFO:main:  Num examples = 1000
INFO:main:  Batch size = 32
Evaluating: 100%|██████████| 32/32 [00:26<00:00,  1.23it/s]


In [161]:
print(eval_loss)
print(predicted_logits)
print(true_labels)

0.9293473623692989
[array([[-7.52214   , -4.1660047 ,  0.8407881 , -8.154163  ],
       [-7.601108  , -4.008359  ,  4.884591  , -4.4919167 ],
       [-4.438669  , -6.727441  , -1.4882613 , -9.353939  ],
       [-7.561746  , -7.5449424 ,  4.169641  , -9.396792  ],
       [-8.856995  , -7.9971466 ,  4.681607  ,  5.392554  ],
       [-0.42608625, -8.608711  , -9.2892885 , -5.9025507 ],
       [ 2.8238668 , -2.6996112 , -4.7693186 ,  3.0902665 ],
       [ 3.2523563 , -8.02197   ,  0.25717777, -7.8390174 ],
       [-8.57853   , -2.111511  ,  0.49052688, -2.932205  ],
       [ 5.4337945 , -8.140071  ,  3.2210968 , -5.5553436 ],
       [-8.040455  , -9.545391  , -1.0688888 , -6.511161  ],
       [ 5.265546  , -7.955804  , -3.804371  , -6.491672  ],
       [-8.34949   , -8.19844   ,  5.917828  , -2.5920548 ],
       [-0.8637111 , -8.277658  , -8.845618  , -8.036783  ],
       [-9.454522  , -0.51533294,  1.6000224 ,  3.3728447 ],
       [ 5.317707  ,  3.7402782 ,  5.0844383 , -5.099071  ],
    

In [162]:
# Concatenate batches
all_predictions = np.concatenate(predicted_logits, axis=0)
all_true_labels = np.concatenate(true_labels, axis=0)

In [163]:
# print(predicted_logits.shape())
print(all_predictions.shape)

(1000, 4)


In [164]:
all_predicted_labels = np.argmax(all_predictions, axis=1)

In [165]:
# flat_predicted_labels = np.concatenate(all_predicted_labels, axis=0)
# flat_true_labels = np.concatenate(all_true_labels, axis=0)

In [166]:
from sklearn.metrics import f1_score, accuracy_score, classification_report

# Calculate the F1 score. Because this is a multi-class problem, we have
# to set the `average` parameter. TODO - What does `micro` do?
f1 = f1_score(all_true_labels, all_predicted_labels, average='micro') 
accuracy = accuracy_score(all_true_labels, all_predicted_labels)
report = classification_report(all_true_labels, all_predicted_labels, zero_division=0, output_dict=True)

print("F1 score: {:.2%}".format(f1))
print(f'Accuracy: {accuracy}')
print(f'Report: {report}')

F1 score: 74.80%
Accuracy: 0.748
Report: {'0': {'precision': 0.7558139534883721, 'recall': 0.7738095238095238, 'f1-score': 0.7647058823529412, 'support': 252}, '1': {'precision': 0.7695473251028807, 'recall': 0.7857142857142857, 'f1-score': 0.7775467775467776, 'support': 238}, '2': {'precision': 0.7423076923076923, 'recall': 0.7310606060606061, 'f1-score': 0.7366412213740459, 'support': 264}, '3': {'precision': 0.7238493723849372, 'recall': 0.7032520325203252, 'f1-score': 0.71340206185567, 'support': 246}, 'accuracy': 0.748, 'macro avg': {'precision': 0.7478795858209706, 'recall': 0.7484591120261852, 'f1-score': 0.7480739857823586, 'support': 1000}, 'weighted avg': {'precision': 0.7476535560294808, 'recall': 0.748, 'f1-score': 0.7477322050683172, 'support': 1000}}
